In [1]:
import pandas as pd

In [2]:
df1 = pd.DataFrame({'date': ['01/08/2021', '02/08/2021', '03/08/2021'],
                    'score' : [2, 4, 6],
                    'perf': [4, 5, 7]})

df2 = pd.DataFrame({'date': ['01/08/2021', '02/08/2021', '03/08/2021'],
                    'score' : [2, 4, 6],
                    'perf': [7, 8, 7]})

In [3]:
!pip install xlsxwriter

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
     |████████████████████████████████| 149 kB 4.0 MB/s eta 0:00:01


In [5]:
class Comparison:
  def __init__(self, df1, df2, vdf):
    self.df1 = df1
    self.df2 = df2
    self.vdf = vdf

  def __str__(self):
    return "\n".join([str(self.df1), str(self.df2), str(self.vdf)])

  def to_excel(self, path):
    writer = pd.ExcelWriter(path, engine='xlsxwriter')

    # Position the dataframes in the worksheet.
    # The hiding of row is a workaround for a bug: 
    # https://github.com/pandas-dev/pandas/issues/27772#issuecomment-925624634
    next_row = 0
    for df in [self.df1, self.df2, self.vdf]:
      df.to_excel(writer, sheet_name='Sheet1', startrow=next_row)  
      writer.sheets['Sheet1'].set_row(next_row + 2, None, None, {'hidden': True})
      # Additional two rows for the column indices
      # And another hidden row above
      next_row += len(df) + 3 
    writer.save()


def compare_table(
    df1,
    df2,
    col_list=None,
    cutoff_score=[2,6],
    groupby=["date",  "score"],
    title=["Table 1", "Table 2"],
    save=None
):
    assert title is None or len(title) == 2, "expecting exactly 2 titles"
    if col_list is None:
        col_list = set.intersection(set(df1.columns), set(df2.columns)) - set(groupby)
    if cutoff_score is not None:
        df1 = df1.loc[df1["score"].isin(cutoff_score)]
        df2 = df2.loc[df2["score"].isin(cutoff_score)]
    
    df1 = df1.set_index(groupby)[col_list]
    df2 = df2.set_index(groupby)[col_list]
    index = [
        item
        for item in df1.index
        if item in df2.index
    ]
    variation_df = (df1.loc[index,col_list] - df2.loc[index,col_list]) / 100.

    def reset_and_add_title(df, title):
      df = df.reset_index()
      df.columns = pd.MultiIndex.from_arrays([[title] * len(df.columns), df.columns])
      return df

    df1 = reset_and_add_title(df1, title[0])
    df2 = reset_and_add_title(df2, title[1])
    variation_df = reset_and_add_title(variation_df, "Variation")

    comparison = Comparison(df1, df2, variation_df)
    
    if save is not None:
        comparison.to_excel(save)
    return comparison   

In [8]:
comparison = (compare_table(
    df1,
    df2,
    title=["CMOR", "COE"],
    col_list=["perf"],
    save="test.xlsx"
))
print(comparison)

         CMOR           
         date score perf
0  01/08/2021     2    4
1  03/08/2021     6    7
          COE           
         date score perf
0  01/08/2021     2    7
1  03/08/2021     6    7
    Variation            
         date score  perf
0  01/08/2021     2 -0.03
1  03/08/2021     6  0.00
